In [1]:
import os
import cv2
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import numpy as np
from PIL import Image
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from torch.utils.data import Dataset, dataloader
import glob

In [ ]:
class PlantSegmentationDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        
        self.image_path = sorted(glob.glob(os.path.join(root_dir, '*', '*.jpg')))

        print(f"Number of images: {len(self.image_path)}")

    def __len__(self):
        return len(self.image_path)
    
    def __getitem__(self, index):
        

SyntaxError: incomplete input (2476651774.py, line 14)